In [1]:
%pylab qt

import cPickle as pickle
import os
import numpy as np
from glob import glob

import helpers.classes as ch
import helpers.features as fh
import helpers.display as dh
import helpers.geometry as gh
import helpers.loading as lh
import helpers.windows as wh
reload(wh)
reload(ch)
reload(fh)
reload(dh)
reload(gh)
reload(lh)
import algorithms.registration as ar
reload(ar)

BASE_PATH = os.getcwd()
print("Current base path: {0}".format(BASE_PATH))
DATA_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_mesh_2/'
TO_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_registered_outline_new/'

ESTIMATORS = [
    {
        'label': 'Procrustes',
        'fn': ar.procrustes
    },
    {
        'label': 'Affine',
        'fn': ar.affine
    },
    {
        'label': 'Similarity',
        'fn': ar.similarity
    },
    {
        'label': 'Projective',
        'fn': ar.projective
    }
]
REFERENCE_ESTIMATORS = [
    {
        'label': 'Extract landmarks by angle',
        'fn': ar.get_marker_using_angles_estimator
    },
    {
        'label': 'Extract landmarks using space partitioning',
        'fn': ar.get_marker_using_space_partitioning_estimator
    },
    {
        'label': 'Use manually set markers',
        'fn': ar.get_manual_markers
    },
    {
        'label': 'Use manually set markers (5-9 only)',
        'fn': ar.get_manual_markers_5_to_9
    },
    {
        'label': 'Evaluated Spline Points',
        'fn': ar.get_spline_points_estimator
    },
    {
        'label': 'Nearest neighbors',
        'fn': ar.get_nearest_neighbor_point_estimator
    }
]

Populating the interactive namespace from numpy and matplotlib
Current base path: /home/stefan/Dropbox/Masterarbeit


In [2]:
def normalize_outline(outline):
    outline['points'], outline['edges'] = gh.extract_outline(outline['points'], outline['simplices'])
    gh.normalize_outline_with_markers(outline)
    return outline
loaded = lh.load_files(DATA_PATH)
loaded = map(normalize_outline, loaded)

helpers/geometry.py:94: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if not None in markers:


In [3]:
window = dh.registration(
    loaded,
    register_fn=ar.estimate_transform,
    estimators=ESTIMATORS,
    reference_estimators=REFERENCE_ESTIMATORS
)

In [8]:
def get_save_format(outline):
    num_points = outline['registered'].shape[0]
    edges = np.zeros((num_points, 2))
    edges[:, 0] = range(0, num_points)
    edges[:, -1] = range(1, num_points+1)
    edges[-1, 1] = 0
    markers = dict([(i,outline['registered_markers'][i-1,:]) for i in range(1, 12)])
    
    outline_to_save = {
        'filename': outline['filename'],
        'done': True,
        'points': outline['registered'],
        'markers': markers,
        'edges': edges
    }
    
    gh.normalize_outline_with_markers(outline_to_save)
    
    return outline_to_save
lh.save_files(TO_PATH, map(get_save_format, window.bones))